In [ ]:
# Install yfinance library
%pip install -q yfinance --upgrade --no-cache-dir
%pip install -q pandas==1.5.3

In [1]:
# Import the necessary libraries
import pandas as pd
import yfinance as yf
import bs4 as bs
import requests
import matplotlib.pyplot as plt
import warnings
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from lib.utils.helpers import get_stock_symbols, extract_numerical_columns, get_snp_companies, get_cols_to_be_divided_by_total_assets
from lib.utils.stocks import get_meta_data
warnings.filterwarnings("ignore")

In [2]:
stocks = pd.read_csv("./lib/quarterly_statements.csv",index_col=0, parse_dates=True)

In [3]:
# getting all top 1500 US companies 
stock_symbols = get_stock_symbols()

In [4]:
stocks = stocks.loc['2021-01-01':]
stocks = stocks.loc[stocks.ticker.isin(stock_symbols)]
stocks['end_of_quarter'] = stocks.index

In [5]:
# meta_df = get_meta_data(stock_symbols=stock_symbols, stocks=stocks)
# meta_df.to_csv('meta_df_2020.csv', index=False)

In [6]:
meta_df = pd.read_csv('meta_df_2020.csv')
meta_df.end_of_quarter = pd.to_datetime(meta_df.end_of_quarter)
df = pd.merge(stocks, meta_df, how='left', on=['ticker', 'end_of_quarter'])

In [7]:
# df.to_csv('stocks_processed_2020.csv', index=False)

In [8]:
df = pd.read_csv('stocks_processed_2020.csv')

In [9]:
# remove commas from thousands, and replace values that have only '-' with nan
tmp = df['end_of_quarter'].copy()
for col in list(df.columns):
  try:
    df.loc[:, col] = df.loc[:, col].str.replace(',', '').replace(r'^-$', np.nan, regex=True)
  except:
    pass
# all percentage columns, remove the percent sign
percentage_columns = [col for col in df.columns if any(df[col].astype(str).str.contains('%'))]
for col in percentage_columns:
  df.loc[:, col] = df.loc[:, col].str.replace('%', '')
df['end_of_quarter'] = tmp

In [10]:
# selecting only numerical features, we need them for normalization, etc. 'increase' is also a numerical, but we dont modify it since it is our target
numerical_columns = extract_numerical_columns(df)

In [11]:
# we take all data since 2021 (we skip earlier years due covid effects) 
df = df.loc[df.end_of_quarter > '2021-01-01']

In [12]:
df['end_of_quarter'] = pd.to_datetime(df['end_of_quarter'])
df['quarter_year'] = df['end_of_quarter'].dt.to_period('Q')

In [13]:
cols_to_be_divided_by_total_assets = get_cols_to_be_divided_by_total_assets()

In [14]:
# converting all numerical values from string to float
df[numerical_columns] = df[numerical_columns].astype('float')

In [15]:
df[cols_to_be_divided_by_total_assets] = df[cols_to_be_divided_by_total_assets].values / df['Total Assets'].values.reshape(-1, 1)

In [16]:
# Mean normalization of all numerical values. As suggested by finance experts, we take the mean and the standard deviation of the industry per quarter for normalization
numerical_columns.remove('increase') # removing the target variable, we dont want to normalize that value
df_mean = df.groupby(['quarter_year', 'industry']).mean()
df_std = df.groupby(['quarter_year', 'industry']).std()
dff = pd.merge(left=df, right=df_mean, how='left', on=['quarter_year', 'industry'], suffixes=["", "_mean"])
dff = pd.merge(left=dff, right=df_std, how='left', on=['quarter_year', 'industry'], suffixes=["", "_std"])

for col in numerical_columns:
  dff[col] = (dff[col] - dff[col + "_mean"]) / dff[col + "_std"]
df = dff

In [17]:
# One hot encoding of industry, we skip encoding of sector since we have almost all NaNs there
one_hot_encoded_industry = pd.get_dummies(df['industry'], prefix='industry')
# one_hot_encoded_sector = pd.get_dummies(df['sector'], prefix='sector')
df = pd.concat([df, one_hot_encoded_industry], axis=1)
df.drop(['industry', 'sector'], axis=1, inplace=True)

In [18]:
df.increase.max(), df.increase.min()

(3.287690168494993, -0.7172177799983862)

In [19]:
# where target variable 'increase' does not exists, remove those rows
df.dropna(axis=0, inplace=True, subset=['increase'])

In [20]:
# train test split (we leave the last quarter for test)
df_test = df.groupby('ticker', as_index=False).first()
df_train = df.loc[~df.end_of_quarter.isin(list(df_test.end_of_quarter.values))]
df_train.shape, df_test.shape

((14971, 569), (1486, 569))

In [21]:
y_train = df_train[['increase', 'ticker']]
y_test = df_test[['increase', 'ticker']]
df_test = df_test.drop(['ticker', 'end_of_quarter', 'increase', 'quarter_year'], axis=1)
df_train = df_train.drop(['ticker', 'end_of_quarter', 'increase',  'quarter_year'], axis=1)

In [22]:
# going from real valued 'increase' to classification problem
y_train['increase_real_values'] = y_train.increase
class_thresholds = [-0.1, 0.0, 0.1, 0.2]
y_train.increase[y_train.increase_real_values >= class_thresholds[3]] = 4
y_train.increase[(y_train.increase_real_values >= class_thresholds[2]) & (y_train.increase_real_values < class_thresholds[3])] = 3
y_train.increase[(y_train.increase_real_values >= class_thresholds[1]) & (y_train.increase_real_values < class_thresholds[2])] = 2
y_train.increase[(y_train.increase_real_values > class_thresholds[0]) & (y_train.increase_real_values < class_thresholds[1])] = 1
y_train.increase[y_train.increase_real_values <= class_thresholds[0]] = 0

In [23]:
class_counts = dict(zip(*np.unique(y_train.increase.values, return_counts=True)))
class_weights = {class_label: len(y_train.increase.values) / (len(np.unique(y_train.increase.values)) * count) for class_label, count in class_counts.items()}
class_counts

{0.0: 3541, 1.0: 3963, 2.0: 3723, 3.0: 2158, 4.0: 1586}

In [24]:
# We fill all NaN values with maxint from numpy. Currently, we are not sure if this is financially correct (?)
df_train.fillna(np.iinfo('int').max, inplace=True)
df_test.fillna(np.iinfo('int').max, inplace=True)

In [25]:
# Assuming you have your features (x) and labels (y) ready, split the data.
X_train, X_test, Y_train, Y_test = train_test_split(df_train, y_train, test_size=0.1, random_state=42)

In [26]:
# We fit a classifier to predict the increase in percentage into the next Q
rf_classifier = RandomForestClassifier(n_estimators=1000, random_state=42, max_depth=20)#, class_weight=class_weights)
# Fit the model to the training data
rf_classifier.fit(X_train, Y_train.increase)

RandomForestClassifier(max_depth=20, n_estimators=1000, random_state=42)

In [27]:
# Predict on the test set just as sanity check
rf_classifier.predict(X_test)
ticker = Y_test.ticker
p = pd.DataFrame({"ticker": ticker, "preds": rf_classifier.predict(X_test), "true": Y_test.increase, "increase": Y_test.increase_real_values})
# Select the top K stocks that we would invest (still this data is older, just for sanity check and to check for underfitting)
K = 10
p.sort_values(by='preds')[::-1].iloc[:K]

,ticker,preds,true,increase
14552,GPS,4.0,3.0,0.171251
11954,AXL,4.0,3.0,0.139092
6410,IBP,4.0,2.0,0.076306
6044,PR,4.0,4.0,0.621429
10714,UPBD,4.0,4.0,0.285551
5798,DECK,4.0,4.0,0.279550
14445,GDEN,4.0,2.0,0.074520
3863,TJX,4.0,3.0,0.118934
11161,ALL,4.0,2.0,0.095479
9878,ACLS,4.0,4.0,0.706394


In [28]:
# Distribution sanity check
pd.DataFrame({"value": p.preds.value_counts().index, "pred_counts": p.preds.value_counts().values, "true_counts": p.true.value_counts().values})

,value,pred_counts,true_counts
0,2.0,446,411
1,1.0,429,359
2,0.0,412,357
3,3.0,106,216
4,4.0,105,155


In [29]:
# Compute accuracy
accuracy = accuracy_score(p.true, p.preds)
print("Accuracy:", accuracy)
print(f'Confusion matrix: \n {confusion_matrix(p.true, p.preds)}')

Accuracy: 0.40186915887850466
Confusion matrix: 
 [[199 104  35  12   9]
 [101 165 112  21  12]
 [ 52  96 167  26  16]
 [ 28  42  91  29  26]
 [ 32  22  41  18  42]]


In [30]:
proba = rf_classifier.predict_proba(df_test.loc[y_test.ticker.isin(get_snp_companies('500'))])[rf_classifier.predict(df_test.loc[y_test.ticker.isin(get_snp_companies('500'))])== np.max(rf_classifier.predict(df_test.loc[y_test.ticker.isin(get_snp_companies('500'))]))][:, -1]
# predicting on the SNP 500 stocks
p = pd.DataFrame({"ticker": y_test.loc[y_test.ticker.isin(get_snp_companies('500'))].ticker, "preds": rf_classifier.predict(df_test.loc[y_test.ticker.isin(get_snp_companies('500'))]), "increase": y_test.increase.loc[y_test.ticker.isin(get_snp_companies('500'))]})
# selecting the top K stocks we want to put money in
p[p.preds == np.max(p.preds.values)]
b = p[p.preds == np.max(p.preds.values)]
b['proba'] = proba
b.sort_values(by='proba')[::-1].iloc[:10]

,ticker,preds,increase,proba
90,ANET,4.0,0.319264,0.308524
1257,STX,4.0,0.040061,0.266858
807,LRCX,4.0,0.228965,0.238151
1218,SMCI,4.0,2.732703,0.229440
853,META,4.0,0.459776,0.229058
1422,WDC,4.0,0.247279,0.225324
290,CMG,4.0,0.295400,0.219753


In [31]:
f'We have gain (positive) or loss (negative) of {b.sort_values(by="proba")[::-1].iloc[:10].increase.mean()  * 100}% on the invested 10 stocks'

'We have gain (positive) or loss (negative) of 61.763554896821006% on the invested 10 stocks'

In [32]:
p['increase_q'] = p.increase
class_thresholds = [-0.1, 0.0, 0.1, 0.2]
p.increase_q[p.increase >= class_thresholds[3]] = 4
p.increase_q[(p.increase >= class_thresholds[2]) & (p.increase_real_values < class_thresholds[3])] = 3
p.increase_q[(p.increase >= class_thresholds[1]) & (p.increase < class_thresholds[2])] = 2
p.increase_q[(p.increase > class_thresholds[0]) & (p.increase < class_thresholds[1])] = 1
p.increase_q[p.increase <= class_thresholds[0]] = 0

In [34]:
# Compute accuracy on test
accuracy = accuracy_score(p.increase_q, p.preds)
print("Accuracy on test set:", accuracy)
print(f'Confusion matrix on test set: \n {confusion_matrix(p.increase_q, p.preds)}')

Accuracy on test set: 0.3587174348697395
Confusion matrix on test set: 
 [[21 15  7  0  0]
 [10 73 38  0  0]
 [25 55 77  2  1]
 [ 8 37 62  2  0]
 [ 7 19 32  2  6]]


##### From here downwards, all the cells are concatenated and we run experiments from 2021-07-01 to today

In [ ]:
quartals = ['2021-07-01', '2021-10-01',
            '2022-01-01', '2022-04-01', '2022-07-01', '2022-10-01',
            '2023-01-01', '2023-04-01', '2023-07-01', '2023-10-01',
            '2024-01-01']
accuracies = {}
for quartal in quartals:
    df = pd.read_csv('stocks_processed_2020.csv')
    # remove commas from thousands, and replace values that have only '-' with nan
    tmp = df['end_of_quarter'].copy()
    for col in list(df.columns):
        try:
            df.loc[:, col] = df.loc[:, col].str.replace(',', '').replace(r'^-$', np.nan, regex=True)
        except:
            pass
    # all percentage columns, remove the percent sign
    percentage_columns = [col for col in df.columns if any(df[col].astype(str).str.contains('%'))]
    for col in percentage_columns:
        df.loc[:, col] = df.loc[:, col].str.replace('%', '')
    df['end_of_quarter'] = tmp
    # selecting only numerical features, we need them for normalization, etc. 'increase' is also a numerical, but we dont modify it since it is our target
    numerical_columns = extract_numerical_columns(df)
    # we take all data since 2021 (we skip earlier years due covid effects) 
    df = df.loc[(df.end_of_quarter > '2020-01-01') & (df.end_of_quarter < quartal)]
    df['end_of_quarter'] = pd.to_datetime(df['end_of_quarter'])
    df['quarter_year'] = df['end_of_quarter'].dt.to_period('Q')
    # cols_to_be_divided_by_total_assets = get_cols_to_be_divided_by_total_assets()
    # converting all numerical values from string to float
    df[numerical_columns] = df[numerical_columns].astype('float')
    # df[cols_to_be_divided_by_total_assets] = df[cols_to_be_divided_by_total_assets].values / df['Total Assets'].values.reshape(-1, 1)
    # Mean normalization of all numerical values. As suggested by finance experts, we take the mean and the standard deviation of the industry per quarter for normalization
    numerical_columns.remove('increase') # removing the target variable, we dont want to normalize that value
    df_mean = df.groupby(['quarter_year', 'industry']).mean()
    df_std = df.groupby(['quarter_year', 'industry']).std()
    dff = pd.merge(left=df, right=df_mean, how='left', on=['quarter_year', 'industry'], suffixes=["", "_mean"])
    dff = pd.merge(left=dff, right=df_std, how='left', on=['quarter_year', 'industry'], suffixes=["", "_std"])

    for col in numerical_columns:
        dff[col] = (dff[col] - dff[col + "_mean"]) / dff[col + "_std"]
    df = dff
    # One hot encoding of industry, we skip encoding of sector since we have almost all NaNs there
    one_hot_encoded_industry = pd.get_dummies(df['industry'], prefix='industry')
    # one_hot_encoded_sector = pd.get_dummies(df['sector'], prefix='sector')
    df = pd.concat([df, one_hot_encoded_industry], axis=1)
    df.drop(['industry', 'sector'], axis=1, inplace=True)
    df.increase.max(), df.increase.min()
    # where target variable 'increase' does not exists, remove those rows
    df.dropna(axis=0, inplace=True, subset=['increase'])
    # train test split (we leave the last quarter for test)
    df_test = df.groupby('ticker', as_index=False).first()
    df_train = df.loc[~df.end_of_quarter.isin(list(df_test.end_of_quarter.values))]
    df_train.shape, df_test.shape
    y_train = df_train[['increase', 'ticker']]
    y_test = df_test[['increase', 'ticker']]
    df_test = df_test.drop(['ticker', 'end_of_quarter', 'increase', 'quarter_year'], axis=1)
    df_train = df_train.drop(['ticker', 'end_of_quarter', 'increase',  'quarter_year'], axis=1)
    # going from real valued 'increase' to classification problem
    y_train['increase_real_values'] = y_train.increase
    class_thresholds = [-0.1, 0.0, 0.1, 0.2]
    y_train.increase[y_train.increase_real_values >= class_thresholds[3]] = 4
    y_train.increase[(y_train.increase_real_values >= class_thresholds[2]) & (y_train.increase_real_values < class_thresholds[3])] = 3
    y_train.increase[(y_train.increase_real_values >= class_thresholds[1]) & (y_train.increase_real_values < class_thresholds[2])] = 2
    y_train.increase[(y_train.increase_real_values > class_thresholds[0]) & (y_train.increase_real_values < class_thresholds[1])] = 1
    y_train.increase[y_train.increase_real_values <= class_thresholds[0]] = 0
    class_counts = dict(zip(*np.unique(y_train.increase.values, return_counts=True)))
    class_weights = {class_label: len(y_train.increase.values) / (len(np.unique(y_train.increase.values)) * count) for class_label, count in class_counts.items()}
    class_counts
    # We fill all NaN values with maxint from numpy. Currently, we are not sure if this is financially correct (?)
    df_train.fillna(np.iinfo('int').max, inplace=True)
    df_test.fillna(np.iinfo('int').max, inplace=True)
    # Assuming you have your features (x) and labels (y) ready, split the data.
    X_train, X_test, Y_train, Y_test = train_test_split(df_train, y_train, test_size=0.1, random_state=42)
    # We fit a classifier to predict the increase in percentage into the next Q
    rf_classifier = RandomForestClassifier(n_estimators=1000, random_state=42, max_depth=20)#, class_weight=class_weights)
    # Fit the model to the training data
    rf_classifier.fit(X_train, Y_train.increase)
    # Predict on the test set just as sanity check
    rf_classifier.predict(X_test)
    ticker = Y_test.ticker
    p = pd.DataFrame({"ticker": ticker, "preds": rf_classifier.predict(X_test), "true": Y_test.increase, "increase": Y_test.increase_real_values})
    # Select the top K stocks that we would invest (still this data is older, just for sanity check and to check for underfitting)
    K = 10
    p.sort_values(by='preds')[::-1].iloc[:K]
    # Distribution sanity check
    # pd.DataFrame({"value": p.preds.value_counts().index, "pred_counts": p.preds.value_counts().values, "true_counts": p.true.value_counts().values})
    # Compute accuracy
    accuracy = accuracy_score(p.true, p.preds)
    
    # print("Accuracy:", accuracy)
    # print(f'Confusion matrix: \n {confusion_matrix(p.true, p.preds)}')
    proba = rf_classifier.predict_proba(df_test.loc[y_test.ticker.isin(get_snp_companies('500'))])[rf_classifier.predict(df_test.loc[y_test.ticker.isin(get_snp_companies('500'))])== np.max(rf_classifier.predict(df_test.loc[y_test.ticker.isin(get_snp_companies('500'))]))][:, -1]
    # predicting on the SNP 500 stocks
    p = pd.DataFrame({"ticker": y_test.loc[y_test.ticker.isin(get_snp_companies('500'))].ticker, "preds": rf_classifier.predict(df_test.loc[y_test.ticker.isin(get_snp_companies('500'))]), "increase": y_test.increase.loc[y_test.ticker.isin(get_snp_companies('500'))]})
    # selecting the top K stocks we want to put money in
    p[p.preds == np.max(p.preds.values)]
    b = p[p.preds == np.max(p.preds.values)]
    b['proba'] = proba
    b.sort_values(by='proba')[::-1].iloc[:10]
    f'We have gain (positive) or loss (negative) of {b.sort_values(by="proba")[::-1].iloc[:10].increase.mean()  * 100}% on the invested 10 stocks'
    p['increase_q'] = p.increase
    class_thresholds = [-0.1, 0.0, 0.1, 0.2]
    p.increase_q[p.increase >= class_thresholds[3]] = 4
    p.increase_q[(p.increase >= class_thresholds[2]) & (p.increase < class_thresholds[3])] = 3
    p.increase_q[(p.increase >= class_thresholds[1]) & (p.increase < class_thresholds[2])] = 2
    p.increase_q[(p.increase > class_thresholds[0]) & (p.increase < class_thresholds[1])] = 1
    p.increase_q[p.increase <= class_thresholds[0]] = 0
    # Compute accuracy on test
    test_accuracy = accuracy_score(p.increase_q, p.preds)
    # print("Accuracy on test set:", test_accuracy)
    accuracies[quartal] = {"val":accuracy, "test": test_accuracy, "gain": f'{b.sort_values(by="proba")[::-1].iloc[:10].increase.mean()  * 100}%'}
    # print(f'Confusion matrix on test set: \n {confusion_matrix(p.increase_q, p.preds)}')
    print(accuracies[quartal])
    print("######################")

In [58]:
suma =  0
capital = 1000
start_capital = capital
for k in accuracies:
    suma += float(accuracies[k]['gain'][:-1])
    capital += capital * float(accuracies[k]['gain'][:-1]) / 100

print("The summed gain over the quartals is: ", suma, "%")
print(f'Starting with {start_capital} USD, over {len(accuracies)} quartals, we will end up with {capital} USD')

suma =  0
capital = 1000
reinvestment_every_quartal = 2100
start_capital = capital
for k in accuracies:
    suma += float(accuracies[k]['gain'][:-1])
    capital += capital * float(accuracies[k]['gain'][:-1]) / 100 + reinvestment_every_quartal

print(f'Starting with {start_capital} USD, over {len(accuracies)} quartals, if we add {reinvestment_every_quartal} USD plus every quartal, we will end up with {capital} USD')
print(f'Out of {capital} USD, our investment accounts {start_capital + reinvestment_every_quartal *len(accuracies)}, while the gain accounts for {capital - (start_capital + reinvestment_every_quartal *len(accuracies))}')



The summed gain over the quartals is:  187.8494084891033 %
Starting with 1000 USD, over 11 quartals, we will end up with 5141.5545480509045 USD
Starting with 1000 USD, over 11 quartals, if we add 2100 USD plus every quartal, we will end up with 63631.69498059065 USD
Out of 63631.69498059065 USD, our investment accounts 24100, while the gain accounts for 39531.69498059065


In [ ]:
importances = rf_classifier.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf_classifier.estimators_], axis=0)

forest_importances = pd.Series(importances, index=list(df_train.columns))
plt.figure(figsize=(16,16))
fig, ax = plt.subplots(figsize=(30,30))
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()
plt.savefig('feature_importances_plot.png')